In [1]:
import pandas as pd
import numpy as np
import calendar
from timeit import default_timer as timer
import datetime as dt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

In [2]:
X=pd.read_csv('top30.csv')

In [4]:
data=pd.read_excel('card transactions.xlsx')

In [5]:
y=data[['Fraud','Date']]

In [6]:
X.drop('Unnamed: 0',axis=1,inplace=True)

In [7]:
X['date']=data['Date']

In [9]:
oot_x=X[X['date']>='2010-11-01']
oot_y=y[y['Date']>='2010-11-01']

In [10]:
X=X[(X['date']>'2010-01-14') & (X['date']<'2010-10-01')]

In [11]:
y=y[(y['Date']>'2010-01-14') & (y['Date']<'2010-10-01')]

In [12]:
y=y.drop('Date',axis=1)
X=X.drop('date',axis=1)

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25,random_state = 1)

In [14]:
from sklearn.preprocessing import MinMaxScaler
mms = MinMaxScaler()
X_train = mms.fit_transform(X_train)

## Neural Net

In [ ]:
from sklearn.neural_network import MLPClassifier
FDR3=pd.DataFrame()
for i in range(10):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25)
    model = MLPClassifier(n_iter_no_change=50,hidden_layer_sizes=(40,40))
    
    model.fit(X_train,y_train)
    
    predictions=model.predict_proba(X_train)[:,1]
    X_train['predicted']=predictions
    X_train['Fraud']=y_train['Fraud']
    topRows = int(round(len(X_train)*0.03))
    temp = X_train.sort_values('predicted',ascending=False).head(topRows)
    needed=temp.loc[:,'Fraud']
    FDR3.loc[i,'trn']=sum(needed)/sum(X_train.loc[:,'Fraud'])
    
    predictions=model.predict_proba(X_test)[:,1]
    X_test['predicted']=predictions
    X_test['Fraud']=y_test['Fraud']
    topRows = int(round(len(X_test)*0.03))
    temp = X_test.sort_values('predicted',ascending=False).head(topRows)
    needed=temp.loc[:,'Fraud']
    FDR3.loc[i,'tst']=sum(needed)/sum(X_test.loc[:,'Fraud'])
    
    predictions=model.predict_proba(oot_x)[:,1]
    oot_x['predicted']=predictions
    oot_x['Fraud']=oot_y['Fraud']
    topRows = int(round(len(oot_x)*0.03))
    temp = oot_x.sort_values('predicted',ascending=False).head(topRows)
    needed=temp.loc[:,'Fraud']
    FDR3.loc[i,'tst']=sum(needed)/sum(oot_x.loc[:,'Fraud'])
    
    print(i,FDR3.loc[i,'trn'],FDR3.loc[i,'tst'],FDR3.loc[i,'tst'])